In [81]:
import pandas as pd
import requests as r
import sqlalchemy
import time
import datetime
pd.options.display.max_columns = None
from sqlalchemy import create_engine

In [2]:
state_df = pd.read_csv("Resources/us_state_population.csv")
vaccination_df =pd.read_csv("Resources/us_state_vaccinations.csv")

In [3]:
vaccination_df.sample(20)

vaccination_df.dropna(how="all")

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.0,377025.0,70861.0,0.15,1.59,7270.0,1.45,7.69,NaN,NaN,NaN,0.207
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222
2,2021-01-14,Alabama,92300.0,435350.0,80480.0,NaN,1.88,NaN,1.64,8.88,8260.0,7083.0,1445.0,0.212
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226
4,2021-01-16,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7557.0,7498.0,1529.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5608,2021-04-03,Wyoming,270862.0,405665.0,161571.0,18.91,46.80,109423.0,27.92,70.09,263.0,3894.0,6728.0,0.668
5609,2021-04-04,Wyoming,286685.0,405665.0,168129.0,20.52,49.53,118773.0,29.05,70.09,15823.0,6092.0,10526.0,0.707
5610,2021-04-05,Wyoming,286966.0,405665.0,168177.0,20.57,49.58,119075.0,29.06,70.09,281.0,3574.0,6175.0,0.707
5611,2021-04-06,Wyoming,288270.0,423725.0,168771.0,20.69,49.81,119735.0,29.16,73.21,1304.0,3758.0,6493.0,0.680


In [4]:
vaccination_df=vaccination_df.dropna()
vaccination_df.sample(10)

vaccination_df["date"]=pd.to_datetime(vaccination_df["date"])
vaccination_df["day"] = vaccination_df["date"].dt.day
vaccination_df["month"] = vaccination_df["date"].dt.month
#vaccination_df.sort_values(by="date",ascending=False).head(60)



In [5]:
# march_df = vaccination_df.loc[(vaccination_df["date"] == "2021-03-01") | (vaccination_df["date"] == "2021-03-31")]
#march_df
#for row in march_df.iterrows(): 
    #if march_df['location'] != march_df['location']:
        #differenceFrame = dataFrameObject.diff(axis=1)



In [6]:
#march_df1 = march_df["total_vaccinations"].diff()
#march_df1
vaccination_df.sample(10)
#vaccination_df.groupby(['location','month']).diff(-1) * (-1)

#So group by months then max day - 1st of the month then make a dataframe for the first of the month then whatever the max of the month is. 
#Also one that does min because the january does not starts the 1st of the month.
#join those two tables back together on state because that is what we are looking for then it is just max - min on day. Then that's the data cleanup. 
#make sure that you sort on state as long as we are doing two different states. 

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used,day,month
151,2021-03-18,Alaska,351093.0,509025.0,213432.0,18.93,47.99,138464.0,29.18,69.58,5106.0,5390.0,7368.0,0.690,18,3
4447,2021-03-14,South Carolina,1583500.0,1935035.0,1017665.0,11.16,30.76,574646.0,19.77,37.58,0.0,33739.0,6553.0,0.818,14,3
2492,2021-04-06,Maryland,3174405.0,4077360.0,2085932.0,20.23,52.51,1222748.0,34.50,67.44,48380.0,62512.0,10340.0,0.779,6,4
4765,2021-01-24,United States,21848655.0,41411550.0,18502131.0,0.97,6.58,3216836.0,5.57,12.47,1310665.0,1122182.0,3380.0,0.528,24,1
4301,2021-01-13,Rhode Island,42028.0,82175.0,36489.0,0.52,3.97,5490.0,3.44,7.76,2460.0,2460.0,2322.0,0.511,13,1
4075,2021-02-14,Pennsylvania,1822327.0,2689425.0,1363250.0,3.54,14.23,453485.0,10.65,21.01,144098.0,59213.0,4625.0,0.678,14,2
1281,2021-03-30,Georgia,3786303.0,5724605.0,2465539.0,12.06,35.66,1280161.0,23.22,53.92,21959.0,72172.0,6798.0,0.661,30,3
1364,2021-03-27,Guam,81649.0,87860.0,49393.0,19.07,49.25,31613.0,29.80,53.00,253.0,1369.0,8259.0,0.929,27,3
5454,2021-01-25,Wisconsin,315876.0,610100.0,266481.0,0.81,5.43,47409.0,4.58,10.48,7586.0,16186.0,2780.0,0.518,25,1
4138,2021-01-22,Puerto Rico,161528.0,337350.0,128284.0,1.01,5.06,32181.0,4.02,10.56,13733.0,8199.0,2567.0,0.479,22,1


In [7]:
#vaccination_df1 = vaccination_df.groupby("location").resample("30D")
#vaccination_df1dsddfdadsfdasdasda

In [52]:
month_df=vaccination_df.groupby('month')['day'].min()
month_df

month
1    12
2     1
3     1
4     1
Name: day, dtype: int64

In [46]:
 vaccination_df.groupby('month')['day'].max()

month
1    31
2    28
3    31
4     7
Name: day, dtype: int64

In [58]:
vaccination_df

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used,day,month
1,2021-01-13,Alabama,84040.0,378975.0,74792.0,0.19,1.71,9245.0,1.53,7.73,5906.0,5906.0,1205.0,0.222,13,1
3,2021-01-15,Alabama,100567.0,444650.0,86956.0,0.28,2.05,13488.0,1.77,9.07,8267.0,7478.0,1525.0,0.226,15,1
7,2021-01-19,Alabama,130795.0,444650.0,114319.0,0.33,2.67,16346.0,2.33,9.07,7557.0,7523.0,1534.0,0.294,19,1
8,2021-01-20,Alabama,139200.0,483275.0,121113.0,0.37,2.84,17956.0,2.47,9.86,8405.0,7880.0,1607.0,0.288,20,1
9,2021-01-21,Alabama,165919.0,493125.0,144429.0,0.44,3.38,21345.0,2.95,10.06,26719.0,10517.0,2145.0,0.336,21,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5608,2021-04-03,Wyoming,270862.0,405665.0,161571.0,18.91,46.80,109423.0,27.92,70.09,263.0,3894.0,6728.0,0.668,3,4
5609,2021-04-04,Wyoming,286685.0,405665.0,168129.0,20.52,49.53,118773.0,29.05,70.09,15823.0,6092.0,10526.0,0.707,4,4
5610,2021-04-05,Wyoming,286966.0,405665.0,168177.0,20.57,49.58,119075.0,29.06,70.09,281.0,3574.0,6175.0,0.707,5,4
5611,2021-04-06,Wyoming,288270.0,423725.0,168771.0,20.69,49.81,119735.0,29.16,73.21,1304.0,3758.0,6493.0,0.680,6,4


In [61]:
first_day=vaccination_df.groupby(["location","month"]).min()

In [62]:
last_day=vaccination_df.groupby(["location","month"]).max()

In [71]:
monthly_df=last_day-first_day
people_vaccinated= monthly_df['people_vaccinated']
people_vaccinated=pd.DataFrame(people_vaccinated)
people_vaccinated

people_vaccinated
location  month                   
Alabama   1               223509.0
          2               348117.0
          3               501830.0
          4               107203.0
Alaska    1                70348.0
...                            ...
Wisconsin 4               189334.0
Wyoming   1                33233.0
          2                51694.0
          3                57030.0
          4                 7965.0

[243 rows x 1 columns]

In [72]:
state_df

,state,name,popestimate2019
0,1,Alabama,4903185
1,2,Alaska,731545
2,4,Arizona,7278717
3,5,Arkansas,3017804
4,6,California,39512223
5,8,Colorado,5758736
6,9,Connecticut,3565287
7,10,Delaware,973764
8,11,District of Columbia,705749
9,12,Florida,21477737


In [74]:
state_df=state_df.rename(columns={'name':'location'})
state_df

,state,location,popestimate2019
0,1,Alabama,4903185
1,2,Alaska,731545
2,4,Arizona,7278717
3,5,Arkansas,3017804
4,6,California,39512223
5,8,Colorado,5758736
6,9,Connecticut,3565287
7,10,Delaware,973764
8,11,District of Columbia,705749
9,12,Florida,21477737


In [76]:
merged_df=people_vaccinated.merge(state_df, on='location')
merged_df

,location,people_vaccinated,state,popestimate2019
0,Alabama,223509.0,1,4903185
1,Alabama,348117.0,1,4903185
2,Alabama,501830.0,1,4903185
3,Alabama,107203.0,1,4903185
4,Alaska,70348.0,2,731545
...,...,...,...,...
199,Wisconsin,189334.0,55,5822434
200,Wyoming,33233.0,56,578759
201,Wyoming,51694.0,56,578759
202,Wyoming,57030.0,56,578759


In [143]:
merged_df=merged_df.rename(columns={'popestimate2019':'population'})
merged_df['per_month_people_vac_p_month']=(merged_df['people_vaccinated']/merged_df['population'])*100

merged_df

,location,people_vaccinated,state,population,per_month_people_vac_p_month
0,Alabama,223509.0,1,4903185,4.558445
1,Alabama,348117.0,1,4903185,7.099814
2,Alabama,501830.0,1,4903185,10.234776
3,Alabama,107203.0,1,4903185,2.186395
4,Alaska,70348.0,2,731545,9.616360
...,...,...,...,...,...
199,Wisconsin,189334.0,55,5822434,3.251802
200,Wyoming,33233.0,56,578759,5.742114
201,Wyoming,51694.0,56,578759,8.931870
202,Wyoming,57030.0,56,578759,9.853842


In [144]:
engine=sqlalchemy.create_engine('sqlite:///vaccines1.db')

In [145]:
table_exists = engine.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='monthly_vacc';")

if len([x for x in table_exists])>0:
    print('we have a table')
    d= engine.execute("SELECT location,people_vaccinated FROM monthly_vacc")
    loc=[]
    people=[]
    
    for i in d:
        loc.append(i[0])
        people.append(i[1])
    rows_to_insert = merged_df.loc[ ~(
                (merged_df['location'].isin(loc)) 
                & (merged_df['people_vaccinated'].isin(people))
               )]
    
       
   
    if not rows_to_insert.empty:
        print('we have some new rows')
        merged_df.to_sql("monthly_vacc", index=False, if_exists="append" ,con=engine)
        
else:
    merged_df.to_sql("monthly_vacc", index=False, con=engine)

In [147]:
pd.read_sql(''' select location,per_month_people_vac_p_month from monthly_vacc;''',con=engine)

,location,per_month_people_vac_p_month
0,Alabama,4.558445
1,Alabama,7.099814
2,Alabama,10.234776
3,Alabama,2.186395
4,Alaska,9.616360
...,...,...
199,Wisconsin,3.251802
200,Wyoming,5.742114
201,Wyoming,8.931870
202,Wyoming,9.853842


In [89]:
# kaggle datasets download -d bumjunkoo/us-vaccination-progress